In [ ]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
from pyspark.sql.types import StructType, StructField, StringType, BinaryType, IntegerType, DoubleType, TimestampType, DateType, LongType
from delta.tables import DeltaTable
from pyspark.sql.utils import AnalysisException
from pyspark.storagelevel import StorageLevel
from typing import Union, Optional
from pyspark.sql.functions import input_file_name

# --- Credenciais AWS ---
accessKeyId = ""
secretAccessKey = ""

# --- Sessão Spark ---
def create_spark_session() -> SparkSession:
    spark = (
        SparkSession
        .builder
        .appName("Gold Zone Streaming")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .enableHiveSupport()
        .getOrCreate()
    )
    
    spark.sparkContext.setLogLevel("WARN")

    conf = spark.sparkContext._jsc.hadoopConfiguration()
    conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
    conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    conf.set("fs.s3a.fast.upload", "true")
    conf.set("fs.s3a.bucket.all.committer.magic.enabled", "true")
    conf.set("fs.s3a.directory.marker.retention", "keep")
    conf.set("spark.driver.extraClassPath", "/usr/local/spark/jars/*")
    conf.set("spark.driver.memory", "8g")
    conf.set("spark.executor.memory", "16g")
    conf.set("fs.s3a.access.key", accessKeyId)
    conf.set("fs.s3a.secret.key", secretAccessKey)

    return spark

spark = create_spark_session()

In [18]:
# Paths
silver_path = "s3a://dev-lab-02-us-east-2-silver/fato_streamings"
gold_path = f"s3a://dev-lab-02-us-east-2-gold/top_artistas"
checkpoint_path = f"s3a://dev-lab-02-us-east-2-gold/checkpoint/fato_streamings"

In [19]:
df_fato_streamings = spark.readStream \
    .format("delta") \
    .option("ignoreChanges", "true") \
    .load(f"{silver_path}")

In [20]:
df_filtrado = df_fato_streamings.filter(df_fato_streamings.flg_feat == False)


df_artistas_top = df_filtrado.groupBy("artista") \
    .agg(F.count("*").alias("qtd_de_streaming")) \
    .orderBy(F.col("qtd_de_streaming").desc()) \
    .limit(10)

df_artistas_top_com_data = df_artistas_top.withColumn("data_atualizacao", F.current_timestamp())

Com o outputMode("complete"), o Spark vai recalcular toda a agregação e sobrescrever os dados da saída (Gold Path) a cada vez.

In [21]:
#.trigger(processingTime="1 minute") \
#.trigger(once=True)

In [ ]:
query  = df_artistas_top_com_data.writeStream \
    .format("delta") \
    .outputMode("complete") \
    .option("checkpointLocation", f"{checkpoint_path}") \
    .option("path", f"{gold_path}") \
    .trigger(processingTime="1 minute") \
    .start()
query.awaitTermination()

## Modos do trigger() no Spark Streaming

No Spark Streaming, o parâmetro 'trigger()' define a frequência com que o micro-batch (janela de dados) é processado.
Existem diferentes tipos de gatilhos (triggers) que controlam a execução do processamento.

#### 1. 'ProcessingTime' ⏱️
- O trigger baseado no tempo de processamento define a frequência fixa com que os batches são executados.
- Você pode especificar o intervalo de tempo (em milissegundos) para o Spark processar os dados em intervalos regulares.
- É o modo padrão, com o valor de 500 milissegundos
Exemplo:

O Spark irá processar os dados a cada 10 segundos, por exemplo.
```python
processing_time_example = """
# Exemplo de código para 'ProcessingTime' ⏱️
streaming_data \
  .writeStream \
  .trigger(processingTime="10 seconds") \
  .outputMode("append") \
  .format("parquet") \
  .option("checkpointLocation", "/path/to/checkpoint") \
  .start("/path/to/output")
"""
```
#### 2. 'Once' 🔄⏲️
- O trigger 'once' fará o Spark processar todos os dados disponíveis no stream uma única vez.
- Após processar os dados, o streaming será encerrado.
- Ideal para processamentos em lote em vez de streaming contínuo.

Exemplo:
O Spark irá processar os dados uma vez e, depois disso, o stream é interrompido.
```python
once_example = """
# Exemplo de código para 'Once' 🔄⏲️
streaming_data \
  .writeStream \
  .trigger(once=True) \
  .outputMode("append") \
  .format("parquet") \
  .option("checkpointLocation", "/path/to/checkpoint") \
  .start("/path/to/output")
"""
```
#### 3. 'Continuous' 🔄⚡
- O trigger 'continuous' permite o processamento de dados de forma contínua com o menor intervalo de tempo possível.
- Ideal para processamento em tempo real com latência muito baixa.
- Requer o modo de execução 'continuous processing' e oferece uma execução quase constante.

Exemplo:
O Spark irá processar os dados assim que eles chegarem, com o mínimo de latência.
```python
continuous_example = """
Exemplo de código para 'Continuous' 🔄⚡
streaming_data \
  .writeStream \
  .trigger(continuous="1 second") \
  .outputMode("append") \
  .format("parquet") \
  .option("checkpointLocation", "/path/to/checkpoint") \
  .start("/path/to/output")
"""
```
#### Resumo:
- **ProcessingTime** ⏱️: processa os dados em intervalos fixos de tempo.
- **Once** 🔄⏲️: processa os dados uma única vez e interrompe o streaming.
- **Continuous** 🔄⚡: processa os dados de forma contínua com latência mínima.

> A escolha do trigger depende das necessidades de processamento em tempo real e do volume de dados.
